`pidgin.rendering` controls the execution of `jinja2` and `mistune`.

In [1]:
    import re, dataclasses, re, functools, IPython, jinja2, inspect, html, collections, pygments, mistune, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    ip = IPython.get_ipython()

In [2]:
    import IPython, base64, nbconvert, jinja2
    with __import__('importnb').Notebook():
        try:
            from . import weave
        except:
            import weave

In [3]:
    mimes = 'text/html image/svg+xml text/latex image/bmp image/png image/jpeg image/gif text/markdown text/javascript application/javascript text/plain'.split()

In [4]:
    def flatten(str): return '&#xA;'.join(str.splitlines())

In [5]:
  def finalize_ipython(object=None, bundle=None):
        if bundle is None:
            bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in mimes:
            if type == 'text/plain': 
                if object is None: return bundle[type]
                return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image') and 'svg' not in type: 
                    object = format_images(type, bundle)._repr_html_()
                if type ==  'text/html':
                    object = flatten(object)
                if type == 'image/svg+xml':
                    object = flatten(re.sub('(xmlns(:?\S*)?="\S*?")?', r'', object))
                if type == 'text/latex':
                    if object.startswith('$$') and object.endswith('$$'): object = object[1:-1]
                break
        return object

In [6]:
    class IPTemplate:
        def render(self, *args, **kwargs):
            """Update the filters and namespace before rendering."""
            if not (args or kwargs): kwargs.update(self.environment.ip.user_ns)
            return super().render(*args, **kwargs)

In [7]:
    def new_environment(ip):
        environment = nbconvert.exporters.TemplateExporter().environment
        environment.finalize = finalize_ipython 
        environment.undefined=jinja2.StrictUndefined
        environment.ip = ip
        environment.template_class = type('ipython_template_class', (IPTemplate, environment.template_class,), {})
        environment.globals.update(vars(__import__('builtins')))
        return environment

In [8]:
    import base64, IPython

    def format_images(type, bundle):
        str = bundle[type]
        if isinstance(str, bytes):
            str = base64.b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'): 
            return IPython.display.HTML(str)
            
        if str.startswith('http'): 
            return F"""<img src="{str}"/>"""

        return F"""<img src="data:{type};base64,{str}"/>"""

In [9]:
    @dataclasses.dataclass
    class PrePopulateFilters:
        ip: None
        def __call__(self): self.ip.environment.filters.update({
            k: v for k, v in self.ip.user_ns.items()
            if (k not in self.ip.environment.filters) and callable(v)
        })

In [10]:
    def load_ipython_extension(ip):
        ip.environment = new_environment(ip)
        ip.events.register('pre_execute', PrePopulateFilters(ip))
    if __name__ == '__main__': load_ipython_extension(get_ipython())

In [11]:
    def unload_ipython_extension(ip):
        try: del ip.environment
        except: ...
        ip.events.unregister('pre_execute', PrePopulateFilters(ip))